# 第 2 章 使用官方 LangChain 套件

In [1]:
import os
import pytz

from typing import List
from openai import OpenAI
from datetime import datetime
from rich import print as pprint

from langchain_openai import ChatOpenAI
from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache
from langchain.prompts import PromptTemplate
from langchain.callbacks import get_openai_callback
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser, PydanticOutputParser, JsonOutputParser
from langchain.prompts import MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate

In [2]:
os.environ['OPENAI_API_KEY'] = "sk-None-vowLahS2p4mOq6FP56VCT3BlbkFJTY1umKuhsfu61iHTNVDc"
timezone = pytz.timezone('Asia/Taipei')

## 2-1 安裝與使用 langchain 套件

### 1. 建立與使用 OpenAI 物件

In [3]:
client = OpenAI(api_key = os.getenv('OPENAI_API_KEY'))
models = client.models.list()
pprint(models)

SyncPage[Model](
    data=[
        Model(id='dall-e-3', created=1698785189, object='model', owned_by='system'),
        Model(id='gpt-4-1106-preview', created=1698957206, object='model', owned_by='system'),
        Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'),
        Model(id='tts-1-hd-1106', created=1699053533, object='model', owned_by='system'),
        Model(id='tts-1-hd', created=1699046015, object='model', owned_by='system'),
        Model(id='gpt-4o-mini-2024-07-18', created=1721172717, object='model', owned_by='system'),
        Model(id='gpt-4-0125-preview', created=1706037612, object='model', owned_by='system'),
        Model(id='babbage-002', created=1692634615, object='model', owned_by='system'),
        Model(id='gpt-4-turbo-preview', created=1706037777, object='model', owned_by='system'),
        Model(id='text-embedding-3-small', created=1705948997, object='model', owned_by='system'),
        Model(id='text-embedding-3-large', created=1705953180, object='model', owned_by='system'),
        Model(id='tts-1', created=1681940951, object='model', owned_by='openai-internal'),
        Model(id='gpt-3.5-turbo', created=1677610602, object='model', owned_by='openai'),
        Model(id='whisper-1', created=1677532384, object='model', owned_by='openai-internal'),
        Model(id='gpt-4o-2024-05-13', created=1715368132, object='model', owned_by='system'),
        Model(id='text-embedding-ada-002', created=1671217299, object='model', owned_by='openai-internal'),
        Model(id='gpt-3.5-turbo-16k', created=1683758102, object='model', owned_by='openai-internal'),
        Model(id='davinci-002', created=1692634301, object='model', owned_by='system'),
        Model(id='gpt-4-turbo-2024-04-09', created=1712601677, object='model', owned_by='system'),
        Model(id='tts-1-1106', created=1699053241, object='model', owned_by='system'),
        Model(id='gpt-3.5-turbo-0125', created=1706048358, object='model', owned_by='system'),
        Model(id='gpt-4-turbo', created=1712361441, object='model', owned_by='system'),
        Model(id='gpt-3.5-turbo-1106', created=1698959748, object='model', owned_by='system'),
        Model(id='gpt-4o-mini', created=1721172741, object='model', owned_by='system'),
        Model(id='gpt-4o', created=1715367049, object='model', owned_by='system'),
        Model(id='gpt-3.5-turbo-instruct-0914', created=1694122472, object='model', owned_by='system'),
        Model(id='gpt-4-0613', created=1686588896, object='model', owned_by='openai'),
        Model(id='gpt-4', created=1687882411, object='model', owned_by='openai'),
        Model(id='gpt-3.5-turbo-instruct', created=1692901427, object='model', owned_by='system')
    ],
    object='list'
)

使用`ChatOpenAI`物件並引入想要使用的模型，並使用`invoke`得到模型的回覆

In [4]:
chat_model = ChatOpenAI(model='gpt-3.5-turbo', api_key=os.environ['OPENAI_API_KEY'], cache=False)
response = chat_model.invoke("你好, 使用繁體中文")
print(response)

content='您好！有什麼可以幫助您的嗎？' response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 19, 'total_tokens': 39}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-d3790913-f41b-47f9-9330-99c19c7579e5-0' usage_metadata={'input_tokens': 19, 'output_tokens': 20, 'total_tokens': 39}


除了`print`之外，還可以使用`pretty_print`和`pprint`來得到美觀的輸出

In [5]:
response.pretty_print()

================================== Ai Message ==================================

您好！有什麼可以幫助您的嗎？


In [6]:
pprint(response)

AIMessage(
    content='您好！有什麼可以幫助您的嗎？',
    response_metadata={
        'token_usage': {'completion_tokens': 20, 'prompt_tokens': 19, 'total_tokens': 39},
        'model_name': 'gpt-3.5-turbo-0125',
        'system_fingerprint': None,
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run-d3790913-f41b-47f9-9330-99c19c7579e5-0',
    usage_metadata={'input_tokens': 19, 'output_tokens': 20, 'total_tokens': 39}
)

### 2. 模型的功能

#### (1) 多筆訊息
可以使用`batch`一次讓模型讀取多筆訊息。從以下的例子也可以知道模型不像ChatGPT一樣有記憶的功能

In [7]:
pprint(chat_model.batch(["我家小狗叫千早愛音", "我家小狗叫什麼？, 使用繁體中文回答"])) # 一次回答兩筆訊息

[
    AIMessage(
        content='，是一只非常可爱的小狗。它有着一身柔软的毛发和大大的眼睛，总是让人忍不住想要抱抱它。千早愛音非常聪
明，学东西也很快，是我们家的得力助手。它喜欢跟着我一起散步，还喜欢跟我一起玩耍。无论我心情好坏，只要看到千早愛音，
就会感到幸福和快乐。我真的很爱我的小狗千早愛音。',
        response_metadata={
            'token_usage': {'completion_tokens': 177, 'prompt_tokens': 22, 'total_tokens': 199},
            'model_name': 'gpt-3.5-turbo-0125',
            'system_fingerprint': None,
            'finish_reason': 'stop',
            'logprobs': None
        },
        id='run-9f45bf17-06b4-45b2-a34a-99a773d88cd8-0',
        usage_metadata={'input_tokens': 22, 'output_tokens': 177, 'total_tokens': 199}
    ),
    AIMessage(
        content='抱歉，我無法回答這個問題，因為我不知道你家小狗叫什麼。你可以告訴我牠的名字嗎？我可以幫你記住。',
        response_metadata={
            'token_usage': {'completion_tokens': 65, 'prompt_tokens': 34, 'total_tokens': 99},
            'model_name': 'gpt-3.5-turbo-0125',
            'system_fingerprint': None,
            'finish_reason': 'stop',
            'logprobs': None
        },
        id='run-beb0aaeb-f8fb-41e5-8b19-81ed20ea9da9-0',
        usage_metadata={'input_tokens': 34, 'output_tokens': 65, 'total_tokens': 99}
    )
]

#### (2) 串流
串流功能可以還原語言模型進行文字接龍的過程

In [8]:
chunks = chat_model.stream("你好")
print(chunks)

<generator object BaseChatModel.stream at 0x761053f47290>


In [9]:
for chunk in chunks:
    pprint(chunk, end="", flush=True)

AIMessageChunk(content='', id='run-26efe557-4a0f-4932-83aa-78da1d20a448')

AIMessageChunk(content='你', id='run-26efe557-4a0f-4932-83aa-78da1d20a448')

AIMessageChunk(content='好', id='run-26efe557-4a0f-4932-83aa-78da1d20a448')

AIMessageChunk(content='！', id='run-26efe557-4a0f-4932-83aa-78da1d20a448')

AIMessageChunk(content='有', id='run-26efe557-4a0f-4932-83aa-78da1d20a448')

...

AIMessageChunk(content='你', id='run-26efe557-4a0f-4932-83aa-78da1d20a448')

AIMessageChunk(content='的', id='run-26efe557-4a0f-4932-83aa-78da1d20a448')

AIMessageChunk(content='吗', id='run-26efe557-4a0f-4932-83aa-78da1d20a448')

AIMessageChunk(content='？', id='run-26efe557-4a0f-4932-83aa-78da1d20a448')

AIMessageChunk(
    content='',
    response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125'},
    id='run-26efe557-4a0f-4932-83aa-78da1d20a448'
)

設定`flush`為`True`來看到跟ChatGPT一樣文字一個一個一個生成的過程

In [10]:
for chunk in chunks:
    print(chunk.content, end="", flush=True)

#### (3) 暫存
設定`cache=True`讓模型使用快取,這樣對模型要求相同內容時就會取回相同結果,可以省錢

In [11]:
chat_model2 = ChatOpenAI(model='gpt-3.5-turbo', api_key=os.getenv('OPENAI_API_KEY'), cache=True)

使用`%%time`計算時間，並且讀取`invoke`出來的回應的id

In [12]:
%%time
set_llm_cache(InMemoryCache())
print(chat_model2.invoke("你好, 使用繁體中文").id)

run-1df27554-1734-4f34-b2a3-1f42d77909e4-0
CPU times: user 4.74 ms, sys: 5.1 ms, total: 9.85 ms
Wall time: 981 ms


再次使用一樣的prompt，發現id一樣，且執行時間變很快。這就是讀取暫存的結果。

In [13]:
%%time
print(chat_model2.invoke("你好, 使用繁體中文").id)

run-1df27554-1734-4f34-b2a3-1f42d77909e4-0
CPU times: user 937 μs, sys: 1.37 ms, total: 2.31 ms
Wall time: 2.37 ms


In [14]:
del chat_model2

#### (4) 計算費用與tokens
單一prompt的示例

In [15]:
with get_openai_callback() as cb:
    result = chat_model.invoke("貴安，使用繁體中文回答")
    pprint(result.content)
    pprint(result.response_metadata['token_usage']) # 計算token數
    pprint(cb) # 計算花費

您好，有什麼我可以幫助您的嗎？

{'completion_tokens': 21, 'prompt_tokens': 23, 'total_tokens': 44}

Tokens Used: 44
        Prompt Tokens: 23
        Completion Tokens: 21
Successful Requests: 1
Total Cost (USD): $4.3e-05

多個prompt的示例

In [16]:
with get_openai_callback() as cb:
    for content in chat_model.batch(["我家小狗叫Soyorin, 使用繁體中文", "我家小狗叫什麼？, 使用繁體中文"]):
        print(content.content)
    pprint(cb)

Soyorin是一隻非常可愛的小狗，非常調皮活潑。它喜歡追逐球玩耍，也喜歡跟其他狗狗一起玩耍。它非常聰明，學東西也很快。我們一家人都非常愛Soyorin，它已經成為我們家的一員。希望Soyorin可以健康快樂的成長，陪伴我們度過每一天。
很抱歉，我不清楚您家小狗叫什麼名字。您可以告訴我它的名字嗎？我很樂意和您聊聊您家可愛的小狗。


Tokens Used: 285
        Prompt Tokens: 60
        Completion Tokens: 225
Successful Requests: 2
Total Cost (USD): $0.00036750000000000004

不同模型花費會不一樣

In [17]:
chat_model.model_name='gpt-4-turbo'
with get_openai_callback() as cb:
    print(chat_model.invoke("貴安，使用繁體中文回答").content)
    pprint(cb)

您好！有什麼我可以幫助您的呢？


Tokens Used: 43
        Prompt Tokens: 23
        Completion Tokens: 20
Successful Requests: 1
Total Cost (USD): $0.00083

### 3. 設定與隱藏金鑰的方法

Colab 提供有 Secret, 可以依帳戶儲存機密資料, 並設定是否允許必記本存取。機密資料不會隨筆記本分享出去, 只有帳戶擁有者才能讀取。

In [18]:
# # 匯入套件
# from google.colab import userdata

In [19]:
# from langchain_openai import ChatOpenAI
# chat_model = ChatOpenAI(model='gpt-3.5-turbo',
#                         api_key=userdata.get('OPENAI_API_KEY'))
# print(chat_model.invoke("你好, 使用繁體中文").content)

## 2-2 與語言模型溝通的藝術 – 提示模板 (PromptTemplate)

### 1. 字串提示模板

字串提示模板主要用於單一對話中的設定。藉由設定提示模板中的參數，下問問類似的問題時只需要指定這些參數就可以了。

In [20]:
prompt_template = PromptTemplate(
    template="請介紹{topic}的知識，使用繁體中文回答",
    input_variables=["topic"])
pprint(prompt_template)

PromptTemplate(input_variables=['topic'], template='請介紹{topic}的知識，使用繁體中文回答')

利用`format`代入字串的方法如下：

In [21]:
prompt_string = prompt_template.format(topic="GPT2")
print(prompt_string)

請介紹GPT2的知識，使用繁體中文回答


利用`invoke`代入字串的方式如下。

In [22]:
prompt_value = prompt_template.invoke({"topic":"GPT2"})
pprint(prompt_value)

StringPromptValue(text='請介紹GPT2的知識，使用繁體中文回答')

兩種方法一個回傳字串，另一個則是回傳`StringPromptValue`物件

In [23]:
print(f"prompt_string： {type(prompt_string)}, \n"
      f"prompt_value： {type(prompt_value)}")

prompt_string： <class 'str'>, 
prompt_value： <class 'langchain_core.prompt_values.StringPromptValue'>


In [24]:
print(chat_model.invoke(prompt_value).content)

GPT-2（Generative Pre-trained Transformer 2）是由OpenAI於2019年開發的一種自然語言處理和生成模型。它是基於Transformer架構的改進版，主要用於處理和生成人類語言。GPT-2的設計目的是透過大量的文本數據進行無監督學習，從而能夠生成連貫且有語境的文本。

### 主要特點

1. **自注意力機制（Self-Attention Mechanism）**：
   GPT-2採用的Transformer模型基於自注意力機制，這使得模型在處理每個詞時能夠關注到整個句子中的所有其他詞。這種機制幫助模型更好地理解語言的上下文關系。

2. **大量的參數**：
   GPT-2擁有多個版本，參數數從1.17億到15.4億不等。這些龐大的參數使得GPT-2擁有強大的語言理解和生成能力。

3. **廣泛的預訓練**：
   在訓練過程中，GPT-2被預訓練在一個非常廣泛的Web文本數據集上（稱為WebText），這包括了從互聯網上抓取的8百萬網頁的文本。這種預訓練幫助模型學習到豐富的語言模式和知識。

4. **無監督學習**：
   GPT-2的訓練過程不依賴於特定的任務導向訓練數據，而是通過預測下一個詞的方式來進行訓練，這屬於一種無監督學習。

### 應用範圍

GPT-2可用於多種自然語言處理任務，如文本生成、機器翻譯、文本摘要、對話系統等。由於其強大的語言生成能力，它在創造看似真實的文本方面表現尤其突出，可以用於創建新聞文章、撰寫故事、生成對話等。

### 安全和倫理考量

由於GPT-2的強大文本生成能力，OpenAI最初選擇不完全公開最大版本的訓練細節和模型數據，以防止潛在的濫用，如生成假新聞或進行社會工程攻擊。這也引發了關於AI技術的倫理和安全的廣泛討論。

總而言之，GPT-2是一個在自然語言處理領域具有重要影響的模型，它的開發不僅推動了語言模型的技術進步，也對AI的倫理和政策制定提出了新的挑戰和考量。


使用`PromptTemplate.from_template`方法來解析字串，就可以直接得到含模板參數的prompt，比較方便

In [25]:
prompt = PromptTemplate.from_template("請介紹{topic}的知識，使用繁體中文回答")
pprint(prompt)

PromptTemplate(input_variables=['topic'], template='請介紹{topic}的知識，使用繁體中文回答')

甚至可以用`+`號連接`PromptTemplate`物件以及含大括號的字串

In [26]:
prompt = PromptTemplate.from_template("請介紹{topic}的知識，使用繁體中文回答") + ", 並以{role}的角度說明"
pprint(prompt)
prompt_value = prompt.invoke({"topic":"GPT2", "role":"國小老師對國小小朋友"})
print(chat_model.invoke(prompt_value).content)

PromptTemplate(
    input_variables=['role', 'topic'],
    template='請介紹{topic}的知識，使用繁體中文回答, 並以{role}的角度說明'
)

好的，小朋友們，今天我們要來認識一個叫做GPT-2的東西。GPT-2是一種很聰明的電腦程式，它可以幫助我們寫文章、回答問題，甚至創造故事哦！

首先，GPT-2的全名是「Generative Pre-trained Transformer 2」，我們可以叫它「生成預訓練轉換器2」。它是由一個叫做OpenAI的公司創造的。這個名字看起來有點複雜，對吧？其實我們可以把它想像成一個很善於學習的機器小助手。

GPT-2非常擅長理解和生成語言，就像是它學過很多書籍、文章，從中學到了如何使用語言。當我們向GPT-2提問或要求它幫忙寫點什麼時，它會根據它所學到的知識來給我們答案或寫出文字。

這個程式是通過一種特別的訓練方式訓練出來的，這種訓練方式叫做「預訓練」。這就像是在學校里，老師會先教我們很多知識，然後我們再用這些知識去做作業或考試。GPT-2也是先學習了大量的文字資料，然後才能幫助我們解決問題。

最後，我們要知道的是，雖然GPT-2很聰明，但它畢竟是機器，有時候也會犯錯，所以我們使用它的時候還是要仔細檢查它給我們的答案是否正確哦！

希望你們對GPT-2有了一點了解。它就像是一個有點超能力的電腦朋友，可以幫助我們學習和完成很多有趣的任務呢！


### 2. 對話提示模板

對話提示模板（`ChatPromptTemplate`）使用於聊天對話。其由三種prompt構成，分別是`SystemMessagePromptTemplate`，`HumanMessagePromptTemplate`，`HumanMessagePromptTemplate`。後續會在介紹這三種prompt是什麼。以下是建立`ChatPromptTemplate`物件的方法

In [27]:
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一位老師很會教{topic}."),
        ("human", "可以再說一次嗎？"),
        ("ai", "好的, 我再講解一次"),
        ("human", "{input}"),
    ]
)
pprint(chat_template)

ChatPromptTemplate(
    input_variables=['input', 'topic'],
    messages=[
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(input_variables=['topic'], template='你是一位老師很會教{topic}.')
        ),
        HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='可以再說一次嗎？')),
        AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='好的, 我再講解一次')),
        HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))
    ]
)

代入參數的方式同樣也可以使用`format_messages`以及`invole`兩種方法

In [28]:
messages_list = chat_template.format_messages(topic="數學", input="什麼是三角函數？")
pprint(messages_list)

[
    SystemMessage(content='你是一位老師很會教數學.'),
    HumanMessage(content='可以再說一次嗎？'),
    AIMessage(content='好的, 我再講解一次'),
    HumanMessage(content='什麼是三角函數？')
]

In [29]:
messages_value = chat_template.invoke({"topic":"數學", "input":"可以再說一次嗎"})
pprint(messages_value)

ChatPromptValue(
    messages=[
        SystemMessage(content='你是一位老師很會教數學.'),
        HumanMessage(content='可以再說一次嗎？'),
        AIMessage(content='好的, 我再講解一次'),
        HumanMessage(content='可以再說一次嗎')
    ]
)

兩者回傳的資料型別也不同，一種是Python的串列，另一種是`ChatPromptValue`物件。

In [30]:
print(f"messages_string： {type(messages_list)}, \n"
      f"messages_value： {type(messages_value)}")

messages_string： <class 'list'>, 
messages_value： <class 'langchain_core.prompt_values.ChatPromptValue'>


不過兩者都可以傳入`invoke`方法得到語言模型的回答

In [31]:
result1 = chat_model.invoke(messages_list).content
result2 = chat_model.invoke(messages_value).content
print(result1)
print(result2)

三角函數是數學中一類重要的函數，主要用於描述與角度相關的比例關係。它們在幾何、三角學、工程、物理學和許多其他科學領域中都非常重要。三角函數通常定義在直角三角形和單位圓上。

在直角三角形中，三角函數定義如下：
1. 正弦函數（sine, 縮寫為 sin）：對於一個角度，正弦值等於該角的對邊長除以斜邊長。
2. 餘弦函數（cosine, 縮寫為 cos）：對於一個角度，餘弦值等於該角的鄰邊長除以斜邊長。
3. 正切函數（tangent, 縮寫為 tan）：對於一個角度，正切值等於該角的對邊長除以鄰邊長。

除此之外，還有割函數（secant, sec）、餘割函數（cosecant, csc）和餘切函數（cotangent, cot），這些是正弦、餘弦和正切的倒數。

三角函數也可以在單位圓上定義。單位圓是一個半徑為1的圓，圓心位於坐標系的原點。在單位圓上，任一角度的正弦值是該角終邊與y軸的交點的y坐標，餘弦值是x坐標，而正切值則是y坐標除以x坐標。

這些函數的圖形是周期性的，意味著它們會重複其形狀。例如，正弦和餘弦函數的周期是360度或2π弧度，正切和餘切函數的周期是180度或π弧度。
當然可以！請問您需要我重複哪部分的內容或解釋哪個數學概念？


### 3. 角色訊息

`SystemMessagePromptTemplate`，`HumanMessagePromptTemplate`，`HumanMessagePromptTemplate`在代入對話提示模板後會轉化成`SystemMessage`，`HumanMessage`以及`AIMessage`物件。因此也可以用直接指定的方式來建立`ChatPromptTemplate`物件，如以下兩種方法

In [32]:
prompt1 = ChatPromptTemplate(messages=[SystemMessage(content="你是一名醫生"), 
                                       HumanMessage(content="我生病了"),
                                       AIMessage(content="哪裡不舒服")])
pprint(prompt1)

ChatPromptTemplate(
    input_variables=[],
    messages=[
        SystemMessage(content='你是一名醫生'),
        HumanMessage(content='我生病了'),
        AIMessage(content='哪裡不舒服')
    ]
)

In [33]:
prompt2 = SystemMessage(content="你是一名醫生") + HumanMessage(content="我生病了") + AIMessage(content="哪裡不舒服")
pprint(prompt2)

ChatPromptTemplate(
    input_variables=[],
    messages=[
        SystemMessage(content='你是一名醫生'),
        HumanMessage(content='我生病了'),
        AIMessage(content='哪裡不舒服')
    ]
)

In [34]:
result1 = chat_model.invoke(prompt1.format_messages()).content
result2 = chat_model.invoke(prompt2.format_messages()).content
print(result1)
print(result2)

我很抱歉聽到你生病了。請描述一下你的症狀，這樣我才能更好地了解你的情況。如果你的症狀嚴重或持續存在，我建議你盡快就醫。
我很抱歉聽到你生病了。請問你有哪些症狀？這樣我才能更好地為你提供一些建議。如果症狀嚴重，我強烈建議你立即就醫。


用指定`SystemMessage`，`HumanMessage`以及`AIMessage`物件建立的`ChatPromptTemplate`物件當然也可以指定模板參數。

In [35]:
prompt = SystemMessage(content="你是一名醫生") +\
         HumanMessage(content="我生病了") +\
         AIMessage(content="哪裡不舒服") +\
         HumanMessagePromptTemplate.from_template("{input}")
pprint(prompt)

ChatPromptTemplate(
    input_variables=['input'],
    messages=[
        SystemMessage(content='你是一名醫生'),
        HumanMessage(content='我生病了'),
        AIMessage(content='哪裡不舒服'),
        HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))
    ]
)

`HumanMessagePromptTemplate`可以直接省略成字串

In [36]:
prompt = SystemMessage(content="你是一名醫生") +\
         HumanMessage(content="我生病了") +\
         AIMessage(content="哪裡不舒服") +\
         "{input}"
pprint(prompt)

ChatPromptTemplate(
    input_variables=['input'],
    messages=[
        SystemMessage(content='你是一名醫生'),
        HumanMessage(content='我生病了'),
        AIMessage(content='哪裡不舒服'),
        HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))
    ]
)

代入模板參數的方式也是使用`format_messages`方法

In [37]:
input_prompt = prompt.format_messages(input="我烙賽")
pprint(input_prompt)
print(chat_model.invoke(input_prompt).content)

[
    SystemMessage(content='你是一名醫生'),
    HumanMessage(content='我生病了'),
    AIMessage(content='哪裡不舒服'),
    HumanMessage(content='我烙賽')
]

您說的「烙賽」可能是指感到非常累或不適吧？如果您能具體描述一下您的症狀，比如頭痛、發熱、肌肉疼痛、呼吸困難等，我可以提供一些基本的建議。不過，請記得最好是直接咨詢醫生進行面對面的診斷和治療。


## 2-3 提示模板的進階變化

### 1. 提示模板參數部分固定
先建立一個有雙參數的prompt

In [38]:
prompt = PromptTemplate(template="試著以{role}的角度, 告訴我一個關於{topic}的知識",
                        input_variables=["role", "topic"])
pprint(prompt)

PromptTemplate(input_variables=['role', 'topic'], template='試著以{role}的角度, 告訴我一個關於{topic}的知識')

使用`prompt.partial`可以產生其中一個參數固定的prompt

In [39]:
partial_prompt1 = prompt.partial(topic="GPT2")
pprint(partial_prompt1)

PromptTemplate(
    input_variables=['role'],
    partial_variables={'topic': 'GPT2'},
    template='試著以{role}的角度, 告訴我一個關於{topic}的知識'
)

也可以使用`PromptTemplate`產生partial prompt

In [40]:
partial_prompt2 = PromptTemplate(template="試著以{role}的角度, 告訴我一個關於{topic}的知識",
                                 input_variables=["role"],
                                 partial_variables={"topic":"GPT2"})
pprint(partial_prompt2)

PromptTemplate(
    input_variables=['role'],
    partial_variables={'topic': 'GPT2'},
    template='試著以{role}的角度, 告訴我一個關於{topic}的知識'
)

同樣利用`format`傳入參數

In [41]:
print(partial_prompt2.format(role="國中老師對國中學生"))

試著以國中老師對國中學生的角度, 告訴我一個關於GPT2的知識


來看看兩者的差異

### 2. `ChatPromptTemplate`模板參數部分固定
先建立一個`ChatPromptTemplate`物件

In [42]:
chat_template = ChatPromptTemplate.from_messages([("system", "試著以{role}的角度說明"),
                                                  ("human", "告訴我一個關於{topic}的知識")])
pprint(chat_template)

ChatPromptTemplate(
    input_variables=['role', 'topic'],
    messages=[
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(input_variables=['role'], template='試著以{role}的角度說明')
        ),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(input_variables=['topic'], template='告訴我一個關於{topic}的知識')
        )
    ]
)

使用`partial`固定其中一個變數，再用`format_messages`傳入另一個變數

In [43]:
chat_partial_prompt = chat_template.partial(topic="GPT2")
pprint(chat_partial_prompt)

ChatPromptTemplate(
    input_variables=['role'],
    partial_variables={'topic': 'GPT2'},
    messages=[
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(input_variables=['role'], template='試著以{role}的角度說明')
        ),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(input_variables=['topic'], template='告訴我一個關於{topic}的知識')
        )
    ]
)

In [44]:
pprint(chat_partial_prompt.format_messages(role="國中老師對國中學生"))

[SystemMessage(content='試著以國中老師對國中學生的角度說明'), HumanMessage(content='告訴我一個關於GPT2的知識')]

### 3. 提示模板加上函式
如果prompt中的參數是另一個（比方說用Python寫的）函式的回傳值，要怎麼辦呢？此處以問LLM現在時間為例

In [45]:
def get_datetime():
    now = datetime.now(timezone)
    return now.strftime("%Y/%m/%d, %H:%M:%S")
print(get_datetime())

2024/08/02, 23:08:41


我們可以使用`partial`方法將`date`固定為`get_datetime`的回傳值

In [46]:
prompt = PromptTemplate.from_template("現在時間是：{date}")
partial_prompt = prompt.partial(date=get_datetime)
pprint(partial_prompt)

PromptTemplate(
    input_variables=[],
    partial_variables={'date': <function get_datetime at 0x7610504da700>},
    template='現在時間是：{date}'
)

In [47]:
pprint(partial_prompt.format())

現在時間是：2024/08/02, 23:08:41

### 4. 提示模板插入訊息
使用`MessagesPlaceholder`物件，我們可以在`ChatPromptTemplate`中建立「內容是一串對話」的變數。例如下列例子中，在`chat_prompt`除了有用之前的方法定義的變數`word_count`外，我們還使用`MessagesPlaceholder`建立了`conversation`變數。之後這個`conversation`變數可以代入一串由`AIMessage`, `HumanMessage`, `SystemMessage`等形成的串列。

In [48]:
human_prompt = "用 {word_count} 個字總結我們迄今為止的對話"
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)
chat_prompt = ChatPromptTemplate.from_messages([MessagesPlaceholder(variable_name="conversation"),
                                                human_message_template])
pprint(chat_prompt)

ChatPromptTemplate(
    input_variables=['conversation', 'word_count'],
    input_types={
        'conversation': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, 
langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, 
langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, 
langchain_core.messages.tool.ToolMessage]]
    },
    messages=[
        MessagesPlaceholder(variable_name='conversation'),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['word_count'],
                template='用 {word_count} 個字總結我們迄今為止的對話'
            )
        )
    ]
)

In [49]:
text = """
真是會虛情假意呢
想演奏是你們的自由，你們就請便吧
到現在都還執著於過去，真難看
你也差不多該忘記了吧
那麼那個樂團算什麼
你講的話和做的事全都互相矛盾
CRYCHIC已經毀了
絕對不可能再復活了
我已經親手將它結束了
沒有人那樣拜托你
這是最後的警告
今後不要再和我扯上關係了
你是抱著多大的覺悟說出這種話的
你只不過是一個學生，有辦法背負其他人的人生嗎
“什麼都願意做”就是這麼沉重的話
做不來的事就別輕易說出口
你這個人，滿腦子都只想到自己呢
"""

使用`format_prompt`代入參數

In [50]:
human_message = HumanMessage(content="請模仿豐川祥子講話")
ai_message = AIMessage(content = text)
new_chat_prompt = chat_prompt.format_prompt(conversation=[human_message, ai_message], word_count="20")
pprint(new_chat_prompt)

ChatPromptValue(
    messages=[
        HumanMessage(content='請模仿豐川祥子講話'),
        AIMessage(
            content='\n真是會虛情假意呢\n想演奏是你們的自由，你們就請便吧\n到現在都還執著於過去，真難看\n你也差不多
該忘記了吧\n那麼那個樂團算什麼\n你講的話和做的事全都互相矛盾\nCRYCHIC已經毀了\n絕對不可能再復活了\n我已經親手將它結
束了\n沒有人那樣拜托你\n這是最後的警告\n今後不要再和我扯上關係了\n你是抱著多大的覺悟說出這種話的\n你只不過是一個學
生，有辦法背負其他人的人生嗎\n“什麼都願意做”就是這麼沉重的話\n做不來的事就別輕易說出口\n你這個人，滿腦子都只想到自
己呢\n'
        ),
        HumanMessage(content='用 20 個字總結我們迄今為止的對話')
    ]
)

看看LLM的回答

In [51]:
print(chat_model.invoke(new_chat_prompt).content)

您請求我模仿豐川祥子的說話風格，我提供了符合其角色的對話範例。


## 2-4 認識輸出內容解析器 (Output Parsers)

### 1. 輸出文字格式的回覆內容
`StrOutputParser`是一個可以將LLM回傳物件的`content`內容取出的物件

In [52]:
str_parser = StrOutputParser()

In [53]:
message = chat_model.invoke("請提供一個國家的名稱和首都, 使用台灣語言")
print("message.content的輸出：", message.content)
print("使用StrOutputParser：", str_parser.invoke(message))

message.content的輸出： 國家的名稱是台灣，首都是台北。
使用StrOutputParser： 國家的名稱是台灣，首都是台北。


### 2. 輸出 JSON 格式的內容
使用`JsonOutputParser`物件將回傳值寫成json格式

In [54]:
json_parser = JsonOutputParser()
format_instructions = json_parser.get_format_instructions()
print(format_instructions)

Return a JSON object.


In [55]:
message = chat_model.invoke("請提供一個國家的名稱和首都,"
                    f"{format_instructions}, 使用台灣語言")
print(message.content)

```json
{
  "國家": "台灣",
  "首都": "台北"
}
```


In [56]:
json_output = json_parser.invoke(message)
print(json_output)

{'國家': '台灣', '首都': '台北'}


在一般的`PromptTemplate`中，可以直接在prompt指定回傳值的格式，LLM看得懂。

In [57]:
prompt = PromptTemplate.from_template("請提供一個國家的名稱、首都和{feature}, {format_instructions}, 使用台灣語言")
prompt = prompt.partial(format_instructions=format_instructions)

In [58]:
message = chat_model.invoke(prompt.invoke({"feature":"知名景點"}))
print(message.content)

```json
{
  "國家": "台灣",
  "首都": "台北",
  "知名景點": "台北101"
}
```


不過考慮倒回傳值本身仍然是字串，還是得使用`JsonOutputParser`物件的`invoke`方法改成Python中的`dict`物件

In [59]:
json_output = json_parser.invoke(message)
print(json_output)
print(type(json_output))

{'國家': '台灣', '首都': '台北', '知名景點': '台北101'}
<class 'dict'>


### 3. 輸出 CSV 格式的內容

使用`CommaSeparatedListOutputParser`物件可以將回傳字串以逗點分開形成list，這可以用來產生csv格式的輸出

In [60]:
list_parser = CommaSeparatedListOutputParser()
print(list_parser.get_format_instructions())

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [61]:
# 建立提示模板
prompt = PromptTemplate.from_template("請說出{city}的知名景點\n{instructions}")\
                       .partial(instructions=list_parser.get_format_instructions())
response = chat_model.invoke(prompt.format(city='沼津'))
print(response.content)

深海水族館,清水町海岸,沼津港,三津浜公園,沼津城址公園,千本松原,富士山ビュースポット


In [62]:
pprint(list_parser.invoke(response))

['深海水族館', '清水町海岸', '沼津港', '三津浜公園', '沼津城址公園', '千本松原', '富士山ビュースポット']

### 4. 自訂輸出格式
利用`langchain_core`模組中的`Pydantic`模組，可以自訂輸出的parser。用`BaseModel`類別可以指定parser的格式。比方說以下的例子，可以使output都以
```
{
    "destination": ...
    "activities": ...
    "budget": ...
    "accommodation": ...
}
```
的格式輸出。宣告時，需要指定裡面每一個屬性的資料型態，並以`Field`說明（給LLM看）這個屬性是要什麼資訊

In [63]:
class TravelPlan(BaseModel):
    destination: str = Field(description="旅遊目的地, 如日本北海道")
    activities: List[str] = Field(description="推薦的活動")
    budget: float = Field(description="預算範圍,單位新台幣")
    accommodation: List[str] = Field(description="住宿選項")

In [64]:
parser = PydanticOutputParser(pydantic_object=TravelPlan)
format_instructions = parser.get_format_instructions()
pprint(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": 
"array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": 
["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"destination": {"title": "Destination", "description": "\u65c5\u904a\u76ee\u7684\u5730, 
\u5982\u65e5\u672c\u5317\u6d77\u9053", "type": "string"}, "activities": {"title": "Activities", "description": 
"\u63a8\u85a6\u7684\u6d3b\u52d5", "type": "array", "items": {"type": "string"}}, "budget": {"title": "Budget", 
"description": "\u9810\u7b97\u7bc4\u570d,\u55ae\u4f4d\u65b0\u53f0\u5e63", "type": "number"}, "accommodation": 
{"title": "Accommodation", "description": "\u4f4f\u5bbf\u9078\u9805", "type": "array", "items": {"type": 
"string"}}}, "required": ["destination", "activities", "budget", "accommodation"]}
```

在prompt中包含進這個格式的說明

In [65]:
prompt = ChatPromptTemplate.from_messages([("system","使用繁體中文並根據使用者要求推薦出適合的旅遊計劃, {format_instructions}"),
                                           ("human","{query}")])
new_prompt = prompt.partial(format_instructions=format_instructions)

In [66]:
user_query = "請幫我安排一個沼津旅遊計畫"
user_prompt = new_prompt.invoke({"query": user_query})
response = chat_model.invoke(user_prompt)
pprint(response.content)

{
  "destination": "沼津",
  "activities": [
    "參觀沼津港深海水族館",
    "漫步於三津浜海灘",
    "享受當地海鮮市場",
    "探訪沼津皇家別墅公園"
  ],
  "budget": 15000,
  "accommodation": [
    "沼津港觀光酒店",
    "沼津市中心商業酒店"
  ]
}

使用`invoke`方法將輸出改成`TravelPlan`物件

In [67]:
parser_output = parser.invoke(response)
pprint(parser_output)
print(type(parser_output))

TravelPlan(
    destination='沼津',
    activities=['參觀沼津港深海水族館', '漫步於三津浜海灘', '享受當地海鮮市場', '探訪沼津皇家別墅公園'],
    budget=15000.0,
    accommodation=['沼津港觀光酒店', '沼津市中心商業酒店']
)

<class '__main__.TravelPlan'>


### 5. 結構化輸出格式
將LLM的回答轉為key-value（Python的字典）也可以用`ResponseSchema`和`StructuredOutputParser`達成。方法是先使用`ResponseSchema`指定輸出格式，再依此建立`StructuredOutputParser`物件。

In [68]:
response_schemas = [ResponseSchema(name="country_data", 
                                   description="請提供包含國家的首都和知名景點的 JSON 物件"),
                    ResponseSchema(name="source",
                                   description="回答答案的根據來源, 例如：來源網站網址",
                                   type="list"),
                    ResponseSchema(name="time",
                                   description="國家建國的時間",
                                   type="YYYY-MM-DD")]
output_parser = StructuredOutputParser(response_schemas=response_schemas)
pprint(output_parser.get_format_instructions())

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing 
"```json" and "```":

```json
{
        "country_data": string  // 請提供包含國家的首都和知名景點的 JSON 物件
        "source": list  // 回答答案的根據來源, 例如：來源網站網址
        "time": YYYY-MM-DD  // 國家建國的時間
}
```

傳入LLM的方法跟之前一樣

In [69]:
format_instructions = output_parser.get_format_instructions()
prompt = ChatPromptTemplate.from_messages([("system","使用台灣語言並回答問題,{format_instructions}"),
                                           ("human","{question}")])
prompt = prompt.partial(format_instructions=format_instructions)

In [70]:
response = chat_model.invoke(prompt.format(question="台灣"))
pprint(output_parser.invoke(response))

{
    'country_data': {'首都': '台北', '知名景點': ['台北101', '國立故宮博物院', '日月潭']},
    'source': ['https://zh.wikipedia.org/wiki/%E5%8F%B0%E7%81%A3'],
    'time': '1912-01-01'
}

既然是字典當然可以用key去查詢value

In [71]:
pprint(output_parser.invoke(response)['country_data'])

{'首都': '台北', '知名景點': ['台北101', '國立故宮博物院', '日月潭']}